# Using Basic Phases

All simulations in OpenPNM eventually require knowing the physical and thermodynamic properties of the fluids and phases.  OpenPNM provides a framework for computing these values in the ``phase`` submodule, along with a *basic* set of functions for predicting the properties pure phases and mixtures. Our policy is to provide a set of reasonable default functions for use as first approximations, without creating a huge amount of extraneous code in OpenPNM.  There are a number of other packages that can be used when more exacting values are required, such as ``chemicals`` and ``cantera``, which can be used within the framework of OpenPNM.

In this notebook, we will be cover the use of the basic ``Phase`` class, the specific classes for common fluids like ``Air`` and ``Water``.

In [1]:
import openpnm as op
import numpy as np

All simulations start by defining/creating a network.  The ``Demo`` class creates a very simple cubic network with an assortment of useful properties included:

In [2]:
pn = op.network.Demo()

## The basic ``Phase`` class

If your simulation is simple, then a simple ``Phase`` object may be sufficient.  It has no predefined models from computing anything, so you have to either assign known values directly (e.g ``water['pore.viscosity'] = 0.001``) or define models that will compute the values you need.  The models can be taken from the ``openpnm.models.phase`` library, or you can write your own.

In [3]:
phase1 = op.phase.Phase(network=pn)
print(phase1)


══════════════════════════════════════════════════════════════════════════════
phase_01 : <openpnm.phase.Phase at 0x7fda06a264a0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.pressure                                                       9 / 9
  2  pore.temperature                                                    9 / 9
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                                9
  2  throat.all                                                             12
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


Note that all ``phase`` objects need to be associated with a network, which is how ``phase1`` knows how many pore (and throat) values to compute...in this case 9 (and 12). 

### Direct assignment of a constant value

The basic ``Phase`` class creates a more or less empty object with only standard temperature and pressure assigned to each pore. In order to use this object for simulations it needs some additional information. For instance, to compute the permeability of ``pn`` we will need the viscosity.  So, let's assign a known value directly for liquid water:

In [4]:
phase1['pore.viscosity'] = 0.001  # Pa.s
phase1['pore.temperature'] = 273.0

*Pro Tip*: When assigning a scalar value to a dictionary key it gets assigned to every pore (or throat).  The result of the above assignment can be seen below:

In [5]:
phase1['pore.viscosity']

array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])

### Using a built-in model

Perhaps you would like to run simulation in the presence of a temperature gradient, and viscosity is a strong function of temperature.  Instead of assigning a constant viscosity, in this case it is better to assign a pore-scale model that OpenPNM will call to compute the viscosity in each pore.  

The ``models`` library in OpenPNM contains some general models which can be used, such as polynomials or linear lines. A 4th order polynomial can be fit to experimental data yielding the following coefficients:

In [6]:
a4 = 5.8543E-11
a3 = -7.6756E-08
a2 = 3.7831E-05
a1 = -8.3156E-03
a0 = 6.8898E-01

These can be used in the the ``op.models.misc.polynomial`` model as follows:

In [7]:
print(phase1['pore.viscosity'])
f = op.models.misc.polynomial
phase1.add_model(propname='pore.viscosity', 
                 model=f,
                 a = (a0, a1, a2, a3, a4),
                 prop='pore.temperature')
print(phase1['pore.viscosity'])
phase1['pore.temperature'] = 300.0 + np.random.rand(pn.Np)*50
phase1.regenerate_models()
print(phase1['pore.viscosity'])

[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001]
[0.00179952 0.00179952 0.00179952 0.00179952 0.00179952 0.00179952
 0.00179952 0.00179952 0.00179952]
[0.00075415 0.00053397 0.0008538  0.00064233 0.00063694 0.00066499
 0.00079701 0.00041567 0.00073816]


In the above block we can see that the values of ``0.001`` are present from the previous assignment, then after we add the model the values are recomputed at the temperature indicated in ``phase1['pore.temperature']``.  To illustrate the point of using a temperature dependent model, we then set the temperature to a random number between 300 and 350 K, the rerun the models at the new temperatures.

### Using a water-specific model

Because water is so common, OpenPNM has some available functions for its properties:

In [8]:
f = op.models.phase.viscosity.water_correlation
phase1.add_model(propname='pore.viscosity',
                 model=f)
print(phase1['pore.viscosity'])

[0.00072708 0.00050169 0.0008307  0.00061035 0.00060479 0.00063382
 0.00077184 0.00037099 0.00071034]


### Writing your own custom model

This subject is explained in detail in another tutorial, but the basic outline is as follows. 

In [9]:
def custom_mu(target, temperature='pore.temperature'):
    T = target[temperature]
    a4 = 5.854E-11
    a3 = -7.676E-08
    a2 = 3.783E-05
    a1 = -8.316E-03
    a0 = 6.890E-01
    mu = a0 + a1*T + a2*T**2 + a3*T**3 + a4*T**4
    return mu

phase1.add_model(propname='pore.viscosity',
                 model=custom_mu)
print(phase1['pore.viscosity'])


[ 4.13198005e-04  1.38156791e-04  5.28491631e-04  2.77830836e-04
  2.71100663e-04  3.05887794e-04  4.63297182e-04 -4.19564449e-05
  3.94286823e-04]


## Specific Classes for Common Fluids

Air, water, and mercury are used commonly enough that OpenPNM not only has pore-scale models for their propertie (i.e. ``op.models.viscosity.water_correlation``, but we have also created pre-defined classes with all the appropriate models already attached:

In [10]:
water = op.phase.Water(network=pn)

ModuleNotFoundError: No module named 'thermo'

In [11]:
print(water)

NameError: name 'water' is not defined

As can be seen in the above print-out, a variety of things have been computed, most of which is coming from a specific pore-scale model.  These can be viewed with:

In [12]:
print(water.models)

NameError: name 'water' is not defined

All of these models are functions of all relevant properties, so we can change the temperature and see the new viscosity:

In [13]:
print(water['pore.viscosity'])
water['pore.temperature'] = 300.0 + np.random.rand(pn.Np)*50
water.regenerate_models()
print(water['pore.viscosity'])

NameError: name 'water' is not defined